In [1]:
import numpy as np
import pandas as pd
# import adata
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import os
import datetime
import argparse
import torch
from utils import *
from torch.utils.data import DataLoader
import episcanpy.api as epi

## Loading data

For more information check this [link](https://openproblems.bio/neurips_docs/about_tasks/task3_joint_embedding/)

In [2]:
# data
# !ls ~/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2
# ! cd ~/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2
! ls /user/leuven/331/vsc33180/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2

openproblems_bmmc_multiome_phase2.censor_dataset.output_mod1.h5ad
openproblems_bmmc_multiome_phase2.censor_dataset.output_mod2.h5ad
openproblems_bmmc_multiome_phase2.censor_dataset.output_solution.h5ad


In [3]:
os.listdir("/user/leuven/331/vsc33180/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2/")

['openproblems_bmmc_multiome_phase2.censor_dataset.output_mod1.h5ad',
 'openproblems_bmmc_multiome_phase2.censor_dataset.output_mod2.h5ad',
 'openproblems_bmmc_multiome_phase2.censor_dataset.output_solution.h5ad']

In [4]:
data_folder_path = "/user/leuven/331/vsc33180/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2/"
files = os.listdir(data_folder_path)
# print(data_folder_path + files[0])
data1 = sc.read_h5ad(data_folder_path + files[0])
data2 = sc.read(data_folder_path + files[1])
# data3 = sc.read(data_folder_path + files[2])

Only considering the two last: ['.output_mod2', '.h5ad'].
Only considering the two last: ['.output_mod2', '.h5ad'].


In [5]:
print(data1)
print(data2)

AnnData object with n_obs × n_vars = 42492 × 13431
    obs: 'batch', 'size_factors'
    var: 'gene_ids', 'feature_types'
    uns: 'dataset_id', 'organism'
    layers: 'counts'
AnnData object with n_obs × n_vars = 42492 × 116490
    obs: 'batch'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'


In [6]:
# print(data1.var["feature_types"])
rna = pd.DataFrame.sparse.from_spmatrix(data1.X) # 42492 x 13431
rna.columns = data1.var.index #data1.var["gene_ids"] -> ENSEMBL IDs

sc.pp.highly_variable_genes(data1, n_top_genes=5000) # Find HGVs
rna = rna.transpose().loc[data1.var["highly_variable"] == True].transpose()
rna.head() # 42492 x 5000

,LINC01128,KLHL17,ISG15,C1orf159,CDK11B,SLC35E2B,CDK11A,PEX10,PLCH2,TPRG1L,...,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AL592183.1
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,1.688304,0.000000,1.688304,0.000000,0.000000,1.688304,0.0
1,0.0,0.0,0.0,0.0,1.161511,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,1.161511,0.000000,0.000000,0.000000,0.000000,0.000000,2.651025,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.344029,0.599516,1.455109,0.344029,0.344029,0.599516,0.802834,0.344029,1.455109,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.355826,2.527316,0.000000,0.000000,0.000000,1.911013,0.000000,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.556855,2.646429,1.054988,0.000000,1.556855,1.054988,0.000000,2.337923,0.0


In [10]:
epi.pp.select_var_feature(data2, nb_features=10000, show=False) # Inplace select 10000 variable regions
atac = pd.DataFrame.sparse.from_spmatrix(data2.X)
atac.columns = data2.var.index
atac.head() # 42492 x 10002

,chr1-629497-630394,chr1-633515-634474,chr1-778276-779191,chr1-827067-827948,chr1-958865-959755,chr1-1013003-1013921,chr1-1019151-1019993,chr1-1032734-1033630,chr1-1059182-1060025,chr1-1068898-1069703,...,chrX-154478646-154479538,chrX-154490306-154491212,chrX-154515887-154516793,chrX-154546772-154547705,chrX-154762342-154763226,chrX-155026531-155027401,chrX-155070906-155071798,chrX-155215991-155216852,GL000219.1-99257-100160,KI270713.1-21434-22336
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Setting up the necessary arguments and functions

In [ ]:
# %tb
# # Model Settings =================================================================================================
# parser = argparse.ArgumentParser(description='Gen-RKM Model')

# parser.add_argument('--N', type=int, default=1000, help='Total # of samples')
# parser.add_argument('--mb_size', type=int, default=300, help='Mini-batch size. See utils.py')
# parser.add_argument('--h_dim', type=int, default=2, help='Dim of latent vector') # shared latent space
# parser.add_argument('--capacity', type=int, default=32, help='Capacity of network. See utils.py') # important in CNNs
# parser.add_argument('--x_fdim', type=int, default=128, help='Input x_fdim. See utils.py') # feature map dimensionality
# parser.add_argument('--y_fdim', type=int, default=20, help='Input y_fdim. See utils.py') # feature map dimensionality
# parser.add_argument('--c_accu', type=float, default=100, help='Input weight on recons_error')

# # Training Settings =============================
# parser.add_argument('--lr', type=float, default=1e-4, help='Input learning rate for optimizer')
# parser.add_argument('--max_epochs', type=int, default=100, help='Input max_epoch for cut-off')
# # parser.add_argument('--device', type=str, default='cuda', help='Device type: cuda or cpu')
# parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu', help='Device type: cuda or cpu')
# parser.add_argument('--workers', type=int, default=0, help='# of workers for dataloader')
# parser.add_argument('--shuffle', type=bool, default=True, help='shuffle dataset: true or false')

# opt = parser.parse_args()
args = {
    "N": rna.shape[0],
#     "mb_size": 300,
    "h_dim": 32,
    "capacity": 32,
    "x_fdim": 128,
    "y_fdim": 256,
    "c_accu": 100,
    "lr": 1e-6,
    "max_epochs": 1,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "workers": 15,
    "shuffle": True,
    "batch_size": 100,
}
opt = args
# print(opt["mb_size"])

In [ ]:
def kPCA(X, Y):
#     print(X)
#     print(Y)
    a = torch.mm(X, torch.t(X)) + torch.mm(Y, torch.t(Y))
    nh1 = a.size(0)
    oneN = torch.div(torch.ones(nh1, nh1), nh1).float().to(opt["device"])
    a = a - torch.mm(oneN, a) - torch.mm(a, oneN) + torch.mm(torch.mm(oneN, a), oneN)  # centering
    print(a)
#     print(a, end="\r")
    h, s, _ = torch.svd(a, some=False)
    return h[:, :opt["h_dim"]], s

In [ ]:
# Energy function
def rkm_loss(output1, X, output2, Y):
    h, s = kPCA(output1, output2)
    U = torch.mm(torch.t(output1), h)
    V = torch.mm(torch.t(output2), h)

    x_tilde = net3(torch.mm(h, torch.t(U))) # NET3
    y_tilde = net4(torch.mm(h, torch.t(V))) # NET4

    # Costs
    f1 = torch.trace(torch.mm(torch.mm(output1, U), torch.t(h))) + torch.trace(
        torch.mm(torch.mm(output2, V), torch.t(h)))
    f2 = 0.5 * torch.trace(torch.mm(h, torch.mm(torch.diag(s[:opt["h_dim"]]), torch.t(h))))
    f3 = 0.5 * ((torch.trace(torch.mm(torch.t(U), U))) + (torch.trace(torch.mm(torch.t(V), V))))
    recon_loss = torch.nn.MSELoss()
    f4 = recon_loss(x_tilde, X) + recon_loss(y_tilde, Y)  # reconstruction loss

    loss = - f1 + f3 + f2 + 0.5 * (- f1 + f3 + f2) ** 2 + opt["c_accu"] * f4  # stabilized loss
    return loss

In [ ]:
class create_dirs:
    """ Creates directories for Checkpoints and saving trained models """

    def __init__(self, ct):
        self.ct = ct
        self.dircp = 'checkpoint.pth_{}.tar'.format(self.ct)
        self.dirout = 'Mul_trained_RKM_{}.tar'.format(self.ct)

    def create(self):
        if not os.path.exists('cp/'):
            os.makedirs('cp/')

        if not os.path.exists('out/'):
            os.makedirs('out/')

    def save_checkpoint(self, state, is_best):
        if is_best:
            torch.save(state, 'cp/{}'.format(self.dircp))

In [ ]:
# https://discuss.pytorch.org/t/dataloader-access-two-items-at-the-same-time/22664
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, dataset1, dataset2):
#         self.dataset1 = dataset1 # datasets should be sorted!
#         self.dataset2 = dataset2

        self.dataset1 = torch.tensor(dataset1.values) # datasets should be sorted!
        self.dataset2 = torch.tensor(dataset2.values)

    def __getitem__(self, index):
#         x1 = self.dataset1.loc[index].to_numpy() #loc, que no iloc (puede que iloc funcione)
#         x2 = self.dataset2.loc[index].to_numpy()
        
        x1 = self.dataset1[index].float()
        x2 = self.dataset2[index].float()

        return x1, x2, index

    def __len__(self):
        return len(self.dataset1) # assuming both datasets have same length
    
# print("Hello")
train_loader = DataLoader(
             ConcatDataset(rna, atac),
             batch_size=args["batch_size"], shuffle=args["shuffle"],
             num_workers=args["workers"]
            )
# print("There")

# initial_time = datetime.datetime.now()
# for i, (datax, datay, index) in enumerate(train_loader):
#     if i==0:
#         initial_time_first_iteration = datetime.datetime.now()
#     print(i)
#     print(datax.shape)
#     print(datay.shape)
#     if i==2:
#         break
        
# print("It took {} min overall.".format(datetime.datetime.now()-initial_time))
# print("It took {} min from the first iteration.".format(datetime.datetime.now()-initial_time_first_iteration))

# num_workers = 5 -> It took 0:08:38.815087 min.
# num_workers = 15 -> It took 0:11:13.346173 min overall - It took 0:01:15.955870 min from the first iteration.
# num_workers = 24 -> It took 0:16:11.308331 min overall - It took 0:02:02.066693 min from the first iteration
# num_workers = 48 -> It took 0:40:50.197775 min overall - It took 0:04:59.826997 min from the first iteration
# Estos 4 de arriba se han hecho con pandas df, los de abajo con tensores
# num_workers = 0 -> It took 0:00:20.585820 min overall - It took 0:00:13.921048 min from the first iteration
# num_workers = 5 -> It took 0:00:35.032580 min overall - It took 0:00:12.420856 min from the first iteration
# num_workers = 15 -> It took 0:00:47.020544 min overall - It took 0:00:05.366418 min from the first iteration
# num_workers = 24 -> It took 0:02:04.082042 min overall - It took 0:00:07.720705 min from the first iteration.
# num_workers = 24 -> It took 0:01:13.212152 min overall - It took 0:00:17.465964 min from the first iteration.
# num_workers = 48 -> It took 0:03:54.132624 min overall - It took 0:00:42.083046 min from the first iteration

## Setting up the NNs

In [ ]:
# class Net1(nn.Module): # Phi 1 - RNA to feature map
#     def __init__(self, input_dim, hidden_dim, args):
#         super(Net1, self).__init__()
# #         c = capacity
#         self.linear1 = nn.Linear(in_features=input_dim, out_features=hidden_dim)
#         self.linear2 = nn.Linear(in_features=hidden_dim, out_features=args["x_fdim"])


#     def forward(self, x):
#         x = self.linear1(x)
#         x = F.leaky_relu(x, negative_slope=0.2)
#         x = self.linear2(x)
# #         x = F.leaky_relu(x, negative_slope=0.2)

#         return x

class Net1(nn.Module): # Phi 1 - RNA to feature map
    def __init__(self, input_dim, hidden_dim_list, args):
        super(Net1, self).__init__()
        
        self.transformations = nn.ModuleList()
        self.transformations.append(nn.Linear(in_features=input_dim, out_features=hidden_dim_list[0]))
        
        for i in range(len(hidden_dim_list)):
            if i == len(hidden_dim_list)-1:
                dim = hidden_dim_list[-1]
                self.transformations.append(nn.Linear(in_features=dim, out_features=args["x_fdim"]))
                
            else:
                in_dim, out_dim = hidden_dim_list[i], hidden_dim_list[i+1]
                self.transformations.append(nn.Linear(in_features=in_dim, out_features=out_dim))
        
    def forward(self, x):
        for i, transform in enumerate(self.transformations):
            if i == len(self.transformations)-1:
                x = transform(x)
                # No leaky relu at the end
                
            else:
                x = transform(x)
                x = F.leaky_relu(x, negative_slope=0.2)
    
    
    
# class Net3(nn.Module):
#     def __init__(self, input_dim, hidden_dim, args): #Psi 1 - Feature map to RNA (input dim = 5000)
#         super(Net3, self).__init__()
#         self.linear2t = nn.Linear(in_features=args["x_fdim"], out_features=hidden_dim)
#         self.linear1t = nn.Linear(in_features=hidden_dim, out_features=input_dim)

#     def forward(self, x):
#         x = self.linear2t(x)
#         x = F.leaky_relu(x, negative_slope=0.2)
#         x = self.linear1t(x)
#         x = F.relu(x) # RNA preprocessed has a lower bound of 0

#         return x
    
class Net3(nn.Module):
    def __init__(self, input_dim, hidden_dim_list, args): #Psi 1 - Feature map to RNA (input dim = 5000)
        super(Net3, self).__init__()
        
        hidden_dim_list.reverse() # Reverse the list to be the inverse of Net 1
        self.reverse_transformations = nn.ModuleList()
        self.reverse_transformations.append(nn.Linear(in_features=args["x_fdim"], out_features=hidden_dim_list[0]))
        
        for i in range(len(hidden_dim_list)):
            if i == len(hidden_dim_list)-1:
                dim = hidden_dim_list[-1]
                self.reverse_transformations.append(nn.Linear(in_features=dim, out_features=input_dim))
                
            else:
                in_dim, out_dim = hidden_dim_list[i], hidden_dim_list[i+1]
                self.reverse_transformations.append(nn.Linear(in_features=in_dim, out_features=out_dim))
        
    def forward(self, x):
        for i, transform in enumerate(self.reverse_transformations):
            if i == len(self.reverse_transformations)-1:
                x = transform(x)
                x = F.relu(x) # RNA preprocessed has a lower bound of 0
                
            else:
                x = transform(x)
                x = F.leaky_relu(x, negative_slope=0.2)

In [ ]:
class Net2(nn.Module): # Phi 2 - ATAC to feature map
    def __init__(self, input_dim, hidden_dim_list, args): 
        super(Net2, self).__init__()
#         c = capacity
        self.transformations = nn.ModuleList()
        self.transformations.append(nn.Linear(in_features=input_dim, out_features=hidden_dim_list[0]))
        
        for i in range(len(hidden_dim_list)):
            if i == len(hidden_dim_list)-1:
                dim = hidden_dim_list[-1]
                self.transformations.append(nn.Linear(in_features=dim, out_features=args["x_fdim"]))
                
            else:
                in_dim, out_dim = hidden_dim_list[i], hidden_dim_list[i+1]
                self.transformations.append(nn.Linear(in_features=in_dim, out_features=out_dim))
        
    def forward(self, x):
        for i, transform in enumerate(self.transformations):
            if i == len(self.transformations)-1:
                x = transform(x)
                # No leaky relu at the end
                
            else:
                x = transform(x)
                x = F.leaky_relu(x, negative_slope=0.2)

# class Net2(nn.Module): # Phi 2 - ATAC to feature map
#     def __init__(self, input_dim, hidden_dim_list, args): 
#         super(Net2, self).__init__()
# #         c = capacity
#         self.linear1 = nn.Linear(in_features=input_dim,out_features=hidden_dim_list[0])
#         self.linear2 = nn.Linear(in_features=hidden_dim_list[0], out_features=hidden_dim_list[1])
#         self.linear3 = nn.Linear(in_features=hidden_dim_list[1], out_features=args["y_fdim"])


#     def forward(self, x):
#         x = self.linear1(x)
#         x = F.leaky_relu(x, negative_slope=0.2)
#         x = self.linear2(x)
#         x = F.leaky_relu(x, negative_slope=0.2)
#         x = self.linear3(x)        
# #         x = F.leaky_relu(x, negative_slope=0.2)

#         return x   

class Net4(nn.Module):
    def __init__(self, input_dim, hidden_dim_list, args): #Psi 2 - Feature map to ATAC (input dim = 116490)
        super(Net4, self).__init__()
    
        hidden_dim_list.reverse() # Reverse the list to be the inverse of Net 2
        self.reverse_transformations = nn.ModuleList()
        self.reverse_transformations.append(nn.Linear(in_features=args["x_fdim"], out_features=hidden_dim_list[0]))
        
        for i in range(len(hidden_dim_list)):
            if i == len(hidden_dim_list)-1:
                dim = hidden_dim_list[-1]
                self.reverse_transformations.append(nn.Linear(in_features=dim, out_features=input_dim))
                
            else:
                in_dim, out_dim = hidden_dim_list[i], hidden_dim_list[i+1]
                self.reverse_transformations.append(nn.Linear(in_features=in_dim, out_features=out_dim))
        
    def forward(self, x):
        for i, transform in enumerate(self.reverse_transformations):
            if i == len(self.reverse_transformations)-1:
                x = transform(x)
                x = torch.sigmoid(x) # ATAC is has binary values
                
            else:
                x = transform(x)
                x = F.leaky_relu(x, negative_slope=0.2)        
        
# class Net4(nn.Module):
#     def __init__(self, input_dim, hidden_dim_list, args): #Psi 2 - Feature map to ATAC (input dim = 116490)
#         super(Net4, self).__init__()        
#         self.linear3t = nn.Linear(in_features=args["y_fdim"], out_features=hidden_dim_list[1])
#         self.linear2t = nn.Linear(in_features=hidden_dim_list[1], out_features=hidden_dim_list[0])
#         self.linear1t = nn.Linear(in_features=hidden_dim_list[0], out_features=input_dim)

#     def forward(self, x):
#         x = self.linear3t(x)
#         x = F.leaky_relu(x, negative_slope=0.2)
#         x = self.linear2t(x)
#         x = F.leaky_relu(x, negative_slope=0.2)
#         x = self.linear1t(x)
#         x = torch.sigmoid(x) # ATAC is has binary values

#         return x    

In [ ]:
# args = {"x_fdim":2}
# hid = [1000, 500, 300, 200, 50, 10]
# inp = 10000
# n2 = Net2(inp, hid, args)
# n4 = Net4(inp, hid, args)
# print(n2.transformations)
# print(n4.reverse_transformations)

In [ ]:
def final_compute(data, net1, net2, kPCA, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    """ Function to compute embeddings of full dataset. """
    # Data is a dictionary with keys RNA and ATAC
#     args["shuffle"] = False
#     xt, _, _ = get_mnist_dataloader(args=args)  # loading data without shuffle
    xt = data["RNA"]
    yt = data["ATAC"]
    xtr = net1(xt.to(args.device))
    ytr = net2(yt.to(args.device)) # This was targets at first, but we don't have labels

    h, s = kPCA(xtr, ytr)
    return torch.mm(torch.t(xtr), h), torch.mm(torch.t(ytr), h), h, s

## Initialize nets and parameters

In [ ]:
# net1 = Net1(input_dim=5000, hidden_dim=1000, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
# net2 = Net2(input_dim=116490, hidden_dim_list=[10000, 2000], args=args).float().to(opt["device"]) # input_dim, hidden_dim_list, args
# net3 = Net3(input_dim=5000, hidden_dim=1000, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
# net4 = Net4(input_dim=116490, hidden_dim_list=[10000, 2000], args=args).float().to(opt["device"]) # input_dim, hidden_dim_list, args
# net4 = Net4(input_dim=116490, hidden_dim_list=[10000, 2000], args=args).double().to(opt["device"])

hidden_list_rna = [3000, 1000, 500, 64]
hidden_list_atac = [50000, 25000, 10000, 5000, 2500, 1000, 500, 64]
net1 = Net1(input_dim=5000, hidden_dim_list=hidden_list_rna, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
net2 = Net2(input_dim=116490, hidden_dim_list=, args=args).float().to(opt["device"]) # input_dim, hidden_dim_list, args
net3 = Net3(input_dim=5000, hidden_dim_list=hidden_list_rna, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
net4 = Net4(input_dim=116490, hidden_dim_list=, args=args).float().to(opt["device"])

params = list(net1.parameters()) + list(net3.parameters()) + list(net2.parameters()) + list(net4.parameters())
optimizer = torch.optim.Adam(params, lr=opt["lr"], weight_decay=0)

## Train

In [ ]:
ct = time.strftime("%Y%m%d-%H%M")
dirs = create_dirs(ct=ct)
dirs.create()

In [ ]:
l_cost = 6  # Costs from where checkpoints will be saved
t = 1
cost = np.inf  # Initialize cost
start = datetime.datetime.now()
while cost > 0.2 and t <= opt["max_epochs"]:  # run epochs until convergence
    avg_loss = 0
    for i, (datax, datay, _) in enumerate(train_loader):
        if i < math.ceil(opt["N"] / opt["batch_size"]):
#             print(f'On step {i}', end="\r")
            print(f'On step {i}')
            datax, datay = datax.float().to(opt["device"]), datay.float().to(opt["device"])
            output1 = net1(datax.float())
            output2 = net2(datay.float())
            loss = rkm_loss(output1, datax, output2, datay).float()

            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(parameters=params, max_norm=1)
            optimizer.step()
            avg_loss += loss.detach().cpu().numpy()
        
        else:
            break
    cost = avg_loss

    # Remember lowest cost and save checkpoint
    is_best = cost < l_cost
    l_cost = min(cost, l_cost)

    dirs.save_checkpoint({
        'epochs': t + 1,
        'net1_state_dict': net1.state_dict(),
        'net3_state_dict': net3.state_dict(),
        'net2_state_dict': net2.state_dict(),
        'net4_state_dict': net4.state_dict(),
        'l_cost': l_cost,
        'optimizer': optimizer.state_dict()}, is_best)
    print(f"Epoch: {t}, Cost: {cost}")
    t += 1
print('Finished Training in: {}. Lowest cost: {}'.format(str(datetime.datetime.now() - start), l_cost))

In [ ]:
# # net1(datax.float())
# output1 = net1(datax.double()).double()
# output2 = net2(datay.double()).double()
# print(output1)
# print(output2)
# loss = rkm_loss(output1, datax, output2, datay).double()
# print(loss)

In [ ]:
# # https://discuss.pytorch.org/t/train-simultaneously-on-two-datasets/649
# # class ConcatDataset(torch.utils.data.Dataset):
# #     def __init__(self, *datasets):
# # #         self.datasets = datasets
# #         self.datasets = [torch.tensor(dataset.values) for dataset in datasets]

# #     def __getitem__(self, i):
# #         return tuple(d[i] for d in self.datasets)

# #     def __len__(self):
# #         return min(len(d) for d in self.datasets)

# # import multiprocessing

# # %time
# # https://discuss.pytorch.org/t/dataloader-access-two-items-at-the-same-time/22664
# class ConcatDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset1, dataset2):
# #         self.dataset1 = dataset1 # datasets should be sorted!
# #         self.dataset2 = dataset2

#         self.dataset1 = torch.tensor(dataset1.values) # datasets should be sorted!
#         self.dataset2 = torch.tensor(dataset2.values)

#     def __getitem__(self, index):
# #         print(index)
# #         print(type(index))
# #         print(type(self.dataset1.loc[index].values))
# #         x1 = self.dataset1.loc[index].to_numpy() #loc, que no iloc (puede que iloc funcione)
# #         x2 = self.dataset2.loc[index].to_numpy()
        
#         x1 = self.dataset1[index]
#         x2 = self.dataset2[index]

#         return x1, x2, index

#     def __len__(self):
#         return len(self.dataset1) # assuming both datasets have same length

# print("Hello")
# train_loader = DataLoader(
#              ConcatDataset(rna, atac),
#              batch_size=args["batch_size"], shuffle=args["shuffle"],
#              num_workers=48 #args["workers"]
#             )
# print("There")

# initial_time = datetime.datetime.now()
# for i, (datax, datay, index) in enumerate(train_loader):
#     if i==0:
#         initial_time_first_iteration = datetime.datetime.now()
#     print(i)
#     print(datax.shape)
#     print(datay.shape)
#     if i==2:
#         break
        
# print("It took {} min overall.".format(datetime.datetime.now()-initial_time))
# print("It took {} min from the first iteration.".format(datetime.datetime.now()-initial_time_first_iteration))

# num_workers = 5 -> It took 0:08:38.815087 min.
# num_workers = 15 -> It took 0:11:13.346173 min overall - It took 0:01:15.955870 min from the first iteration.
# num_workers = 24 -> It took 0:16:11.308331 min overall - It took 0:02:02.066693 min from the first iteration
# num_workers = 48 -> It took 0:40:50.197775 min overall - It took 0:04:59.826997 min from the first iteration
# Estos 4 de arriba se han hecho con pandas df, los de abajo con tensores
# num_workers = 0 -> It took 0:00:20.585820 min overall - It took 0:00:13.921048 min from the first iteration
# num_workers = 5 -> It took 0:00:35.032580 min overall - It took 0:00:12.420856 min from the first iteration
# num_workers = 15 -> It took 0:00:47.020544 min overall - It took 0:00:05.366418 min from the first iteration
# num_workers = 24 -> It took 0:02:04.082042 min overall - It took 0:00:07.720705 min from the first iteration.
# num_workers = 24 -> It took 0:01:13.212152 min overall - It took 0:00:17.465964 min from the first iteration.
# num_workers = 48 -> It took 0:03:54.132624 min overall - It took 0:00:42.083046 min from the first iteration